In [1]:
import telebot
import pandas as pd

from gtts import gTTS

In [37]:
class MachineStates:
    """
    класс реализует машину состояний для пользователя
    Количество развилок: 1
    
    Первая развилка: выбор между путем для документа или для череды вопросов
    
    Дорога для документа:
        - Запросить документ
            - проверить правильность формата документа и данных в нем
        
        - Если все верно, сделать предсказание
        - Передать результат предсказания
        - Поставить ссылку на развилку 1
    
    Дорога для вопросов:
        - Вопрос 1
            - Проверка правильности ответа
            - Если все ок, перевести на следующий вопрос
            - Если что-то не верно, указать на ошибку и остаться на текущем вопросе
        - Вопрос 2:
            - Аналогично
        
        ....
        
        
        - Если все верно, сделать предсказание
        - Передать результат предсказания
        - Поставить ссылку на развилку 1

    """
    # Инициализация
    def __init__(self):
        # ссылка на развилку
        self.__next_state = self.choice_way_ask
        # индикатор, нужен ли выбор
        self.need_choice = False

        
    # вызов следующего шага
    def __call__(self, msg=None):
        return self.__next_state(msg)

    
    #_______________________________________________________________________________
    #    Choosing a way
    #_______________________________________________________________________________
        
    # Предоставление выбора (реализовать кнопки)
    def choice_way_ask(self, msg=None):
        self.__next_state = self.choice_way
        self.need_choice = True
        self.features = []
        
        return 'Документ или вопросы?'

    # В зависимости от выбора, запустить дорогу
    def choice_way(self, choice):
        
        if choice.lower() == 'документ':
            self.__next_state = self.start_documnet            
        else:
            self.__next_state = self.start_quiz
        
        requirement = self.__next_state()
        self.need_choice = False

        return 'Выбор сделан.\n' + requirement

    
    #_______________________________________________________________________________
    #    Documnet way
    #_______________________________________________________________________________

    # начало дороги документа
    def start_documnet(self):
        self.__next_state = self.documnet_first_step
        return 'Пришлите пожалуйста документ в формате csv. Должно быть какое-то количество признаков для одного наблюдения'

    
    # Запрашивание документа и его проверка на правильность
    def documnet_first_step(self, doc):
        def check_step(ans):
            # если найдена ошибка
            if ans=='error':
                return 'Ошибка: ты попутал. В такой-то фиче должен быть текст,а не число\n'

            return True
        
        checked_step = check_step(doc)
        # если документ верный и данные в нем адекватны, то меняем ссылку на предсказывателя и вызываем
        if isinstance(checked_step, bool):
            self.features = doc
            self.__next_state = self.get_result
            return self.__next_state()
            
        else:
            return checked_step + 'Пришлите пожалуйста документ в формате csv. Должно быть какое-то количество признаков для одного наблюдения'



    #_______________________________________________________________________________
    #    Quiz way
    #_______________________________________________________________________________
        
    # первый вопрос и перевод на первый шаг
    def start_quiz(self):
        self.__next_state = self.quiz_first_step
        return 'Какой у Вас возраст?'

        
    # проверка ответа на предыдущий вопрос
    def quiz_first_step(self, answer):
        # проверка ответа
        def check_step(ans):
            # если найдена ошибка
            if ans=='error':
                return 'Ошибка в ответе на вопрос №1: ты попутал тип данных. Должно быть число,а не текст.\n'
            return True
        
        checked_step = check_step(answer)
        # если ответ на предыдущий вопрос подходит, то меняем ссылку на следующий шаг и задаем следующий вопрос
        if isinstance(checked_step, bool):
            # добавление фичи
            self.features.append(answer)
            # перевод ссылки на следующий шаг
            self.__next_state = self.quiz_second_step
            return 'Какая пицца Вам больше нравится?'
        # Иначе выводим, почему ответ не принимается и переспрашиваем
        else:
            return checked_step + 'Какой у Вас возраст?'


    # проверка ответа на предыдущий вопрос и если все ок, запрос результата
    def quiz_second_step(self, answer):
        # проверка ответа
        def check_step(ans):
            # если найдена ошибка
            if ans=='error':
                return 'Ошибка в ответе на вопрос №2: ты попутал тип данных. Должен быть текст,а не число.\n'
            return True
        
        
        checked_step = check_step(answer)
        # если ответ на предыдущий вопрос подходит, то меняем ссылку на следующий шаг и задаем следующий вопрос
        if isinstance(checked_step, bool):
            # добавление фичи
            self.features.append(answer) 
            # перевод ссылки на следующий шаг            
            self.__next_state = self.get_result
            # вызов предсказания
            return self.__next_state()
        # Иначе выводим, почему ответ не принимается и переспрашиваем
        else:
            return checked_step + 'Какая пицца Вам больше нравится?'
    
        
    def get_result(self):
        # предсказание на основе self.features : одномерный массив фичей
        submision = '0' 
        # зачищаю признаковое пространство
        self.features = []
        # перевожу ссылку на развилку1
        self.__next_state = self.choice_way_ask
        # отдаю результат
        return 'Хорошо, ваш результат ' + submision


In [38]:
def creat_audio(text):
    tts = gTTS(text, lang='ru', slow=False)
    tts.save('sound_ru.mp3')

In [39]:
def get_predict(data):
    return '0'

IDS_machine_state = {}

#Сюда подставляете свой токен
bot = telebot.TeleBot('5358515008:AAGVmnGBgtnr7Prnm48VwVyeE9bniigr_Fk')

@bot.message_handler(commands=['start'])
def start_message(message):

    bot.send_message(message.chat.id, 'Пришли мне Emoji, который необходимо сделать размытым.')


@bot.message_handler(content_types=['document'])
def handle_docs_photo(message):
    chat_id = message.chat.id
    file_info = bot.get_file(message.document.file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    print(downloaded_file)
    
    
    bot.reply_to(message,  message.document.file_name)
    

@bot.message_handler(content_types=['text'])
def send_text(message='text'):
    global IDS_machine_state
    
    chat_id = message.chat.id 
    # создание машины состояний для нового (после перезапуска) пользователя
    if chat_id not in IDS_machine_state:
        IDS_machine_state[chat_id] = MachineStates()
    

    answer = IDS_machine_state[chat_id](message.text)
    
    #bot.send_message(chat_id, answer)
    creat_audio(answer)
    with open('sound_ru.mp3', 'rb') as audio:
        bot.send_voice(message.chat.id, audio)

    

bot.polling()
#сами смайлики прописать не могу, иначе публикация не обрабатывается на habr